[![Open In Wolfram Cloud](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/PR/wolfram-badge.svg)](https://www.wolframcloud.com/obj/gvarnavi/Published/02X_strange-attractors.nb)

# Strange Attractors

## Chaos Theory

We begin by investigating Chaos, and how we can impose certain symmetries on it.

In [ ]:
Entity["MathWorld", "Chaos"]["BasicDefinitions"][[1]]

In [ ]:
Entity["MathWorld", "Chaos"]["LongTypesetDescription"]

### Chaos Game
Well, let's at-least see how it relates to the fractal systems we've been looking at:

We'll start by choosing a regular polygon with n sides

In [ ]:
GraphicsRow[Graphics[RegularPolygon[#]]&/@Range[3,5]]

Starting from a random point, we iterate using the following algorithm:
- Pick one of the polygon vertices at random
- Travel a fraction of the distance to that vertex
- Repeat

In [ ]:
chaosGame[{n_Integer?Positive, r_}, pts_ : 1000, drop_ : 10] := 
Module[{v, sc = 1., p, i, p0},
  (*Random Starting point*)
  p0 = RandomReal[{-1, 1}, 2];
  (*Vertices*)
  v = CirclePoints[n];
  (*Iteration*)
  p = NestList[r Plus[#, RandomChoice[v] ] &, p0, pts + drop];
  (*Drop first couple of iterations*)
  Drop[p, drop]];

In [ ]:
triangle = Graphics@Point@chaosGame[{3, 0.5}, 5000];
square = Graphics@Point@chaosGame[{4, 0.4}, 5000];
pentagon = Graphics@Point@chaosGame[{5, 0.3}, 5000];
GraphicsRow[{triangle, square, pentagon}, ImageSize -> Large]

## Strange Attractors
>"[They're] a little like old Uncle Jake, who is a bit eccentric. You're not really surprised by what Uncle Jake does, but it's still difficult to understand why he does what he does. But by calling him eccentric, you feel comfortable with his actions"  
~ Symmetry in Chaos, Field and Golubitsky

### De-Jong Attractor
The De Jong Attractor is given by the following formula:

$$\begin{aligned}
x_{n+1} &= sin(a y_n) - cos(b x_n)\\
y_{n+1} &= sin(c x_n) - cos(d y_n)
\end{aligned}
$$

In [ ]:
naiveDeJong[{x_, y_}, {a_, b_, c_, d_}] :=
 {Sin[a y] - Cos[b  x], 
  Sin[c x] - Cos[d  y]}

In [ ]:
naiveDeJong[{1., 1.}, {1, 2, 3, 4}]

In [ ]:
naiveDeJong[%, {1, 2, 3, 4}]

In [ ]:
Graphics[Point[
  NestList[naiveDeJong[#, {1.641, 1.902, 0.316, 1.525}] &, {1., 1.}, 
   100]],ImageSize->Small]

In [ ]:
Graphics[Point[
  NestList[naiveDeJong[#, {1.641, 1.902, 0.316, 1.525}] &, {1., 1.}, 
   10000]],ImageSize->Small]

#### Binning and Coloring
- High-level implementation

In [ ]:
ArrayPlot[
BinCounts[
  NestList[naiveDeJong[#, {1.641, 1.902, 0.316, 1.525}] &, {1., 1.}, 
   10^5], {-2, 2, 0.005}, {-2, 2, 0.005}],ImageSize->Small]

This is very faint, we could either:
- Increase the number of iterations
- Make our bins 'fatter' (i.e. more hits per bin)
- Appy a non-linear smoothing function to the hits

In [ ]:
ArrayPlot[
Log[(BinCounts[
    NestList[naiveDeJong[#, {1.641, 1.902, 0.316, 1.525}] &, {1., 1.},
      10^5], {-2, 2, 0.005}, {-2, 2, 0.005}] + 1)],ImageSize->Small]

This is both slow, and memory-intensive, since we need to store all the points in memory.

#### Compiled Binning
Instead, we can compile in C, binning along the way:

In [ ]:
dejongCompiled = 
 Compile[
 {{xmin, _Real}, {xmax, _Real}, {ymin, _Real}, {ymax, _Real},{delta, _Real}, 
 {itmax, _Integer}, {a, _Real, 0}, {b, _Real, 0}, {c, _Real, 0}, {d, _Real, 0}},
 
  Block[{bins, dimx, dimy, x, y, tx, ty}, 
   bins = ConstantArray[0, Floor[{xmax - xmin, ymax - ymin}/delta] + {1, 1}];  
   {dimx, dimy} = Dimensions[bins];
   
   {x, y} = {0., 0.};
   Do[
   {x, y} = {Sin[a y] - Cos[b x], Sin[c x] - Cos[d y]};
   
    tx = Floor[(x - xmin)/delta] + 1;
    ty = Floor[(y - ymin)/delta] + 1;
    
    If[tx >= 1 && tx <= dimx && ty >= 1 && ty <= dimy, 
     bins[[tx, ty]] += 1
     
     ], {i, 1, itmax}];
     
   bins]
   , CompilationTarget :> "C", RuntimeOptions -> "Speed"]

In [ ]:
ArrayPlot[
 Log[dejongCompiled[-2., 2., -2., 2., 0.005, 10000000, 1.641, 1.902, 
    0.316, 1.525] + 1], Frame -> False, ImageSize -> Small]

Let's add some colour!

In [ ]:
ArrayPlot[
 Log[dejongCompiled[-2., 2., -2., 2., 0.005, 10000000, 1.641, 1.902, 
    0.316, 1.525] + 1], Frame -> False, ImageSize -> Small, 
 ColorFunction -> (ColorData["SunsetColors"][1 - #] &)]

### Imposing Dihedral Symmetry
Here, we'll investigate the particular case of Dihedral symmetry and quote some iterating functions we'll use.  
In the Wallpaper groups notebook later today, we'll derive a more general method to extend this.

#### $D_n$ Symmetries
Dihedral symmetry (also known as polygon symmetry) is satisfied by two operations:
- Reflection along k
- Rotation R by 360/n degrees

It's easier to use complex notation.
Remember how multiplication by a complex number of magnitude 1 and phase 360/n implies a clockwise R rotation.  
Taking k to be the real x-axis, then we can write Dihedral symmetry operations as:
$$
\begin{aligned}
k(z) &= \bar{z} \\
R(z) &= \rho z
\end{aligned}
$$

##### Symmetry-imposing logistic mapping
The logistic map is given by:
$$G(z) = \lambda z (1-z \bar{z})$$

In [ ]:
Graphics[{PointSize[0], 
  Point[Flatten[
    Table[Thread[{r, 
       Nest[r # (1 - #) &, Range[0, 1, 0.01], 1000]}], 
       {r, 0, 4,0.01}], 1]]},ImageSize->Small]

We seek to find a correction to G(z), such that when added to make F(z) we obtain:

$$
\begin{aligned}
F(k(z)) = k(F(z)) \\
F(R(z)) = R(F(z))
\end{aligned}
$$

We'll start by checking the simplest function with Dihedral symmetry is given by $\bar{z}^{n-1}$

In [ ]:
ansatz[z_] := Conjugate[z]^(n - 1)
kappa[z_] := Conjugate[z]

In [ ]:
ansatz[kappa[z]]
kappa[ansatz[z]]

These don't look the same at first glance.  
This is because the Wolfram Language makes no assumptions on the nature of z and n (i.e. real, complex, etc). We'll use Complex Expand, which assumes all the symbols are Real:

In [ ]:
ansatz[kappa[z]] // ComplexExpand
kappa[ansatz[z]] // ComplexExpand
SameQ[% == %%]

Now let's check the rotation, R.  
We'll use polar coordinates to show this:

In [ ]:
rotatePolar[{r_, theta_}, n_] := {r, theta + (360 Degree)/n}
ansatzPolar[{r_, theta_}, n_] := {r^(n - 1), -(n - 1) theta}

In [ ]:
ansatzPolar[rotatePolar[{r, \[Theta]}, n], n]
rotatePolar[ansatzPolar[{r, \[Theta]}, n], n]

The magnitudes look the same, let's make sure the phases are too:

In [ ]:
Simplify[
ansatzPolar[rotatePolar[{r, \[Theta]}, n], n][[2]]
-rotatePolar[ansatzPolar[{r, \[Theta]}, n], n][[2]]
]

Perfect, our symmetrized loggistic equation can therefore be corrected to read:
$$F(z) = \lambda z (1-z \bar{z}) + \gamma \bar{z}^{n-1}$$

### Complex and non-linear mappings
We can keep playing this game to find more complex mappings which preserve symmetry.  
Here, we list two general implementations (from Symmetry in Chaos, Field and Golubitsky), We'll look at more general implementations later:

$$
\begin{aligned}
F_{complex}(z) &= \left( \lambda + \alpha z \bar{z} + \beta \Re[z^n]\right)z + \gamma \bar{z}^{n-1} \\
F_{nonlinear}(z) &= \left( \lambda + \alpha z \bar{z} + \beta \Re[z^n] + \delta \Re[\left(\frac{z}{|z|}\right)^{np}]|z|\right)z + \gamma \bar{z}^{n-1}
\end{aligned}
$$

In [ ]:
attractorComplex = Compile[{
   {xmin, _Real}, {xmax, _Real}, {ymin, _Real}, {ymax, _Real},{delta, _Real},
   {itmax, _Integer}, {n, _Integer},
   {lambda, _Real}, {a, _Real}, {b, _Real}, {c, _Real}, {omega,_Real}}, 
   
   Block[{bins, dim, x, y, tx, ty, z},
   
   bins = ConstantArray[0, Floor[{xmax - xmin, ymax - ymin}/delta] + {1, 1}];
   dim = Dimensions[bins];
   z = -0.3 + 0.2 I;
   
   Do[
    z = (lambda + a z Conjugate[z] + b Re[z^n] + omega I) z + c Conjugate[z]^(n - 1);
    x = Re[z]; y = Im[z];
    
    tx = Floor[(x - xmin)/delta] + 1;
    ty = Floor[(y - ymin)/delta] + 1;
    If[tx >= 1 && tx <= dim[[1]] && ty >= 1 && ty <= dim[[2]], 
     bins[[tx, ty]] += 1];
    
    z, {i, 1, itmax}];
   bins], CompilationTarget -> "C", RuntimeOptions -> "Speed"]
   
   
attractorNonLinear = 
 Compile[{{xmin, _Real}, {xmax, _Real}, {ymin, _Real}, {ymax, _Real},
{delta, _Real}, {itmax, _Integer}, {n, _Integer}, {lambda, _Real},
{a, _Real}, {b, _Real}, {c, _Real}, {d, _Real}, {p, _Integer}}, 
  Block[{bins, dim, x, y, tx, ty, z, b1, radii, normed, coordinates}, 
   
   bins = ConstantArray[0, Floor[{xmax - xmin, ymax - ymin}/delta] + {1, 1}];
   dim = Dimensions[bins];
   z = -0.3 + 0.2 I;
   
   Do[
    z = (lambda + a z Conjugate[z] + b Re[z^n] + d Re[(z/Abs[z])^n p] Abs[z]) z + c Conjugate[z]^(n - 1);
    x = Re[z]; y = Im[z];
    
    tx = Floor[(x - xmin)/delta] + 1;
    ty = Floor[(y - ymin)/delta] + 1;
    If[tx >= 1 && tx <= dim[[1]] && ty >= 1 && ty <= dim[[2]], 
     bins[[tx, ty]] += 1];
    
    z, {i, 1, itmax}];
   bins], CompilationTarget -> "C", RuntimeOptions -> "Speed"]

#### Gallery
- Complex Attractors

In [ ]:
AbsoluteTiming[
 bins = N[attractorComplex[-1., 1., -1., 1., 0.001, 5 10^6, 6, -2.7,5.0, 1.5, 1.0, 0.0]];]
ArrayPlot[Log[bins + 1],ColorFunction -> (ColorData["SunsetColors", #] &),ImageSize->300]

In [ ]:
AbsoluteTiming[bins = N[attractorComplex[-1.5, 1.5, -1.5, 1.5, 0.001, 5 10^6,7, -2.08, 1.0, -0.1, 0.167, 0.0]];]
ArrayPlot[Log[bins + 1],ColorFunction -> (ColorData["SunsetColors", #] &),ImageSize->300]

- Nonlinear Attractors

In [ ]:
AbsoluteTiming[bins = N[attractorNonLinear[-1.5, 1.5, -1.5, 1.5, 0.001, 5 10^6, 3, 1., -2.1, 0.0, 1.0, 1.0, 1]];]
ArrayPlot[Log[bins + 1], ColorFunction -> (ColorData["AvocadoColors", #] &),ImageSize->300]

In [ ]:
AbsoluteTiming[bins = N[attractorNonLinear[-1., 1., -1., 1., 0.001, 5 10^6, 9, -2.5,4.5, -0.7, 1.0, -0.9, 0]];]
ArrayPlot[Log[bins + 1],ColorFunction -> (ColorData["AvocadoColors", #] &),ImageSize->300]

In [ ]:
AbsoluteTiming[ bins = N[attractorNonLinear[-.5, .5, -.5, .5, 0.001, 5 10^6, 5, -2.38, 10.0, -12.3, 0.75, 0.02, 1]];]
ArrayPlot[Log[bins + 1], ColorFunction -> (ColorData["AvocadoColors", #] &),ImageSize->300]

In [ ]:
AbsoluteTiming[bins = N[attractorNonLinear[-1.6, 1.6, -1.6, 1.6, 0.001, 10^7, 6, -2.42, 1.0, -0.04, 0.14, 0.088, 0]];]
ArrayPlot[Log[bins + 1],ColorFunction -> (ColorData["AvocadoColors", #] &),ImageSize->300]

In [ ]:
AbsoluteTiming[bins = N[attractorNonLinear[-1.5, 1.5, -1.5, 1.5, 0.001, 5 10^6, 3, 1.455, -1.0, 0.03, -0.8, -0.025, 0]];]
ArrayPlot[Log[bins + 1], ColorFunction -> (ColorData["AvocadoColors", #] &),ImageSize->300]

### Quilts (Tiling the plane)
Chaotic attractors can be designed to look pleasing when tiled together.  
The trick is to design a pattern without seams between tiles, Mathematically, this is equivalent to patterning on a torus.

This can be done by building a pattern out of elements that fit nicely within the tile. Sinusoids, with their characteristic frequencies, make nice building blocks.

The attractor rules for patterning onto a torus consist of two Fourier expansions:

$$
\begin{aligned}
x_{n+1} = \sum_{m\geq0 \;\&\; n\geq>0} a_{m,n}*sin(2 \pi m x_n)*cos(2 \pi n y_n)\\
y_{n+1} = \sum_{m\geq0 \;\&\; n\geq>0} b_{m,n}*sin(2 \pi m x_n)*cos(2 \pi n y_n)
\end{aligned}
$$

For example, we can build an attractor using 4 such terms on a square lattice:

In [ ]:
squarePattern[alpha_, beta_, gamma_, epsilon_, m_, v_][{x_, y_}] := 
 m*{x, y} + v + alpha*{Sin[2*Pi*x], Sin[2*Pi*y]}  (*m=1,n=0*)+ 
  beta*{Sin[2*Pi*x]*Cos[2*Pi*y], Sin[2*Pi*y]*Cos[2*Pi*x]} (*m=1,n=1*)+
   gamma*{Sin[4*Pi*x], Sin[4*Pi*y]} (*m=2,n=0*)+ 
    epsilon*{Sin[6*Pi*x]*Cos[4*Pi*y], Sin[6*Pi*y]*Cos[4*Pi*x]} (*m=3,n=2*)

In [ ]:
data = NestList[squarePattern[0.25, -0.3, 0.2, 0.3, 1, {0, 0}], 
   RandomReal[{0.2, 0.7}, 2], 1000000];
   
dataBinCounts = BinCounts[data, 0.001, 0.001];
ArrayPlot[Log[dataBinCounts + 1], Frame -> False, 
 ColorFunction -> "SunsetColors",ImageSize->300]

This is rather slow, let's compile in C again:

In [ ]:
attractorTiling = 
 Compile[{{xmin, _Real}, {xmax, _Real}, {ymin, _Real}, {ymax, _Real}, 
{delta, _Real}, {itmax, _Integer}, {lambda, _Real}, 
{a, _Real}, {b,_Real}, {c, _Real}, {m, _Real}, {v, _Real, 1}}, 
  Block[{bins, dim, x, y, tx, ty, z}, 
   bins = ConstantArray[0,Floor[{xmax - xmin, ymax - ymin}/delta] + {1, 1}];
   dim = Dimensions[bins];
   {x, y} = {0.3, 0.2};
   
   Do[{x, y} = {
   m x + v[[1]] + lambda Sin[2 Pi x] + a (Sin[2 Pi x] Cos[2 Pi y]) + b Sin[4 Pi x] + c (Sin[6 Pi x] Cos [4 Pi y]), 
      m y + v[[2]] + lambda Sin[2 Pi y] + a ( Sin[2 Pi y] Cos[2 Pi x]) + b Sin[4 Pi y] + c ( Sin[6 Pi y] Cos [4 Pi x])
      };
    
    (*bining*)
    tx = Floor[(x - xmin)/delta] + 1;
    ty = Floor[(y - ymin)/delta] + 1;
    If[tx >= 1 && tx <= dim[[1]] && ty >= 1 && ty <= dim[[2]], 
     bins[[tx, ty]] += 1], {i, 1, itmax}]; 
     bins],
     
  CompilationTarget -> "C", RuntimeOptions -> "Speed"]

In [ ]:
ArrayPlot[
 Log[attractorTiling[0, 1, 0, 1, 0.001, 3000000, 0.25, -.3, .2, .3,1, {0, 0}] + 1], 
    ColorFunction -> ColorData["SunsetColors"],Frame -> False, ImageSize->300]

Finally, we want to assemble our quilts:

In [ ]:
Block[{
bins = 
   N[attractorTiling[0, 1, 0, 1, 0.003,3000000, 0.25, -.3, .2, .3, 1, {0, 0}]]},
 ArrayPlot[Log[PadRight[bins, {1002, 1002}, bins] + 1], ImageSize -> 300, Frame -> False, 
  ColorFunction -> (ColorData["SunsetColors", #^1.5] &)]]